In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px

from tensorflow.keras.applications import (
    VGG16,
    ResNet50,
    InceptionV3,
    MobileNetV2,
)
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-05-09 11:04:07.217088: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 11:04:07.217208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 11:04:07.358315: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

KeyboardInterrupt



In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
os.listdir('/kaggle/input/100-bird-species')

In [ ]:
train = '/kaggle/input/100-bird-species/train'
validation = '/kaggle/input/100-bird-species/valid'
test = '/kaggle/input/100-bird-species/test'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255) 

valid_datagen = ImageDataGenerator(rescale = 1.0/255.)  

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [ ]:
batch_size = 32

train_generator = train_datagen.flow_from_directory(train, target_size=(224,224),batch_size=batch_size,class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(validation, target_size=(224,224),batch_size=batch_size,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test, target_size=(224,224),batch_size=batch_size,class_mode='categorical')

In [ ]:
n_categories = len(os.listdir('/kaggle/input/100-bird-species/train'))
print(n_categories)

In [ ]:
res_model = tf.keras.applications. MobileNetV2(weights='imagenet',
                                            include_top=False,
                                            input_shape=(224,224,3))

In [ ]:
for layer in res_model.layers[:98]:
    layer.trainable = False
    # Check the freezed was done ok
for i, layer in enumerate(res_model.layers):
    print(i, layer.name, "-", layer.trainable)

In [ ]:

layer_output = res_model.output

x = Flatten()(layer_output)
x = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
x = Dense(n_categories, activation='softmax')(x)

In [ ]:
model = Model(inputs=res_model.input, outputs=x)

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, learning_rate):
    if epoch < 5:
        return learning_rate
    else:
        return learning_rate * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[callback]
)

In [ ]:
model.save("/kaggle/working/my_model.h5")  

In [ ]:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('TL_InceptionV3 Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('TL_InceptionV3 Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
results = pd.DataFrame(history.history)
results.tail()

In [ ]:
X_test, y_test = test_generator.next()

In [ ]:
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Predict labels for test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert multilabel-indicator y_test to single label format
y_test_single_label = np.argmax(y_test, axis=1)

# Calculate F1 score
f1 = f1_score(y_test_single_label, y_pred_classes, average='weighted')

print("F1 Score:", f1)
